In [1]:
import pandas as pd
import numpy as np
# from sklearn.preprocessing import OneHotEncoder
# import time
import warnings
warnings.filterwarnings('ignore')

# enc = OneHotEncoder()
path = 'data/'
train = pd.read_csv(path+'train.csv')
vali = pd.read_csv(path+'validation.csv')
test = pd.read_csv(path+'test.csv')
# start_time = time.time()

In [2]:
basectr = train['click'].value_counts(normalize = True)[1]
basectr

0.0007539649884458758

In [3]:
ad_ctr = []
ad_name = [1458,3476,3427,3358,2259,2821,3386,2997,2261]
for i in ad_name:
    ad_ctr.append(train['click'][train['advertiser']==i].value_counts(normalize = True)[1])
ad_len = len(ad_name)
ad_ctr

[0.00083473226564105031,
 0.00050464562342478474,
 0.00074884754565216914,
 0.00076448084204447765,
 0.00030658545558598702,
 0.00062225602378400803,
 0.0007180766777520589,
 0.0046066034099877034,
 0.00030675100937663219]

In [ ]:
ad_train = []
for i in ad_name:
    ad_train.append(train[train['advertiser']==i])

In [ ]:
ad_imp = [0]*9
for i in range(len(ad_name)):
    if ad_train[i]['bidprice']>ad_train[i]['payprice']:
        ad_imp

In [4]:
train_onehot = pd.read_csv('train_onehot_new.csv')
vali_onehot = pd.read_csv('vali_onehot_new.csv')

In [5]:
vali_ad_onehot = []
for i in ad_name:
    vali_ad_onehot.append(vali_onehot[vali_onehot["advertiser_"+str(i)]==1]) 

In [6]:
def dataprocessing (data,data_onehot):
    slotprice = data['slotprice']
    bidprice = data['bidprice']
    payprice = data['payprice']
    click = data['click']
    
    ad_onehot = []
    
    ad_onehot = []
    for i in ad_name:
        ad_onehot.append(data_onehot[data_onehot["advertiser_"+str(i)]==1]) 
    
    ad_click = []
    for i in ad_name:
        ad_click.append(data['click'][data['advertiser']==i])
        
    return ad_onehot, ad_click, slotprice, bidprice, payprice

In [7]:
train_ad_onehot,train_ad_click,train_slotprice,train_bidprice,train_payprice = dataprocessing(train,train_onehot)
vali_ad_onehot,vali_ad_click,vali_slotprice,vali_bidprice,vali_payprice = dataprocessing(vali,vali_onehot)

In [8]:
len(train_ad_onehot)

9

In [9]:
from sklearn.metrics import mean_squared_error
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
from sklearn.metrics import roc_auc_score as auc
from sklearn.metrics import classification_report as report

In [10]:
train_click = train[['click','advertiser']]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_onehot, train_click, test_size=0.2, random_state=42)

In [11]:
def dataprocessing_ (data_onehot,click):
    ad_onehot = []
    for i in ad_name:
        ad_onehot.append(data_onehot[data_onehot["advertiser_"+str(i)]==1]) 
#     print('done')
    ad_click = []
    for i in ad_name:
#         print(type(click))
        a = click[click['advertiser']==i]
        a = a.drop('advertiser',1)
#         print(a.head())
        ad_click.append(a)    
    return ad_onehot, ad_click

In [12]:
_train_ad_onehot,_train_ad_click = dataprocessing_(X_train,y_train)
_ad_onehot,_ad_click= dataprocessing_(X_test,y_test)

In [13]:
import xgboost as xgb

xgb_model = []
train_ad_pd = []
train_ad_rmse = []
train_ad_auc = []
train_ad_report = []
vali_ad_pd = []
vali_ad_rmse = []
vali_ad_auc = []

param = {'max_depth':4, 'eta':1, 'silent':1, 'objective':'binary:logistic' }
param['nthread'] = 4
param['eval_metric'] = 'auc'
plst = param.items()

num_round = 10

for i in range(ad_len):
    dtrain = (xgb.DMatrix(_train_ad_onehot[i],_train_ad_click[i]))
    dvali = (xgb.DMatrix(_ad_onehot[i],_ad_click[i]))
    evallist  = [(dvali,'eval'), (dtrain,'train')]
    dptrain = xgb.DMatrix(train_ad_onehot[i])
    dpvali =xgb.DMatrix(vali_ad_onehot[i])
    
    xgb_model.append(xgb.train( plst, dtrain, num_round, evallist ))
    train_ad_pd.append(xgb_model[i].predict(dptrain))
    train_ad_rmse.append(rmse(train_ad_click[i],train_ad_pd[i]))
    train_ad_auc.append(auc(train_ad_click[i],train_ad_pd[i]))
    
    vali_ad_pd.append(xgb_model[i].predict(dpvali))
    vali_ad_rmse.append(rmse(vali_ad_click[i],vali_ad_pd[i]))
    vali_ad_auc.append(auc(vali_ad_click[i],vali_ad_pd[i]))
    print(i+1,'/9')

C:\Users\hannah\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[0]	eval-auc:0.874758	train-auc:0.838705
[1]	eval-auc:0.874798	train-auc:0.838751
[2]	eval-auc:0.893077	train-auc:0.87243
[3]	eval-auc:0.924506	train-auc:0.912023
[4]	eval-auc:0.950281	train-auc:0.943449
[5]	eval-auc:0.952527	train-auc:0.946198
[6]	eval-auc:0.959691	train-auc:0.950253
[7]	eval-auc:0.950639	train-auc:0.959192
[8]	eval-auc:0.945097	train-auc:0.962278
[9]	eval-auc:0.945891	train-auc:0.966897
1 /9
[0]	eval-auc:0.5	train-auc:0.5
[1]	eval-auc:0.72314	train-auc:0.829555
[2]	eval-auc:0.723045	train-auc:0.831138
[3]	eval-auc:0.731639	train-auc:0.839872
[4]	eval-auc:0.728257	train-auc:0.866595
[5]	eval-auc:0.727669	train-auc:0.87222
[6]	eval-auc:0.770678	train-auc:0.913726
[7]	eval-auc:0.807829	train-auc:0.935333
[8]	eval-auc:0.8127	train-auc:0.946567
[9]	eval-auc:0.800477	train-auc:0.955399
2 /9
[0]	eval-auc:0.777004	train-auc:0.821562
[1]	eval-auc:0.777025	train-auc:0.821728
[2]	eval-auc:0.795067	train-auc:0.852231
[3]	eval-auc:0.797141	train-auc:0.853403
[4]	eval-auc:0.879431

In [14]:
j = 0
print('xgb trainning results')
for i in ad_name:
    
    print('ad',i,':','RMSE',train_ad_rmse[j],'AUC',train_ad_auc[j])
    
    j+=1
j = 0
print('xgb validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_rmse[j],'AUC',vali_ad_auc[j])
    j+=1

xgb trainning results
ad 1458 : RMSE 0.0210262659299 AUC 0.96357488059
ad 3476 : RMSE 0.0207153351128 AUC 0.925217669784
ad 3427 : RMSE 0.0222728394306 AUC 0.957090440669
ad 3358 : RMSE 0.0236658000896 AUC 0.956338287431
ad 2259 : RMSE 0.0174646943302 AUC 0.865710356755
ad 2821 : RMSE 0.0248399425182 AUC 0.777989154065
ad 3386 : RMSE 0.0264984011767 AUC 0.782049242606
ad 2997 : RMSE 0.0674131482699 AUC 0.741621132551
ad 2261 : RMSE 0.0174405504593 AUC 0.800851119817
xgb validation results
ad 1458 : RMSE 0.0218420981693 AUC 0.966008170071
ad 3476 : RMSE 0.0184966660166 AUC 0.797117514433
ad 3427 : RMSE 0.0245907426539 AUC 0.970605310094
ad 3358 : RMSE 0.0227024799446 AUC 0.926340903666
ad 2259 : RMSE 0.0259096584936 AUC 0.606347790435
ad 2821 : RMSE 0.0250477005773 AUC 0.537762043254
ad 3386 : RMSE 0.0246328486995 AUC 0.746631058697
ad 2997 : RMSE 0.0654375197595 AUC 0.701964688108
ad 2261 : RMSE 0.0193934779049 AUC 0.621129816685


In [15]:
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor,AdaBoostRegressor,GradientBoostingRegressor

In [64]:
from sklearn.ensemble import RandomForestClassifier
rf_model = []
train_ad_rfpd = []
train_ad_rfrmse = []
train_ad_rfauc = []
vali_ad_rfpd = []
vali_ad_rfrmse = []
vali_ad_rfauc = []

for i in range(ad_len):
    rf_model.append(RandomForestClassifier(n_estimators=50, max_depth=15, random_state=2017))
    rf_model[i].fit(train_ad_onehot[i], train_ad_click[i])
    train_ad_rfpd.append((rf_model[i].predict_proba(train_ad_onehot[i]))[:,1])
    train_ad_rfrmse.append(rmse(train_ad_click[i],train_ad_rfpd[i]))
    train_ad_rfauc.append(auc(train_ad_click[i],train_ad_rfpd[i]))
    vali_ad_rfpd.append((rf_model[i].predict_proba(vali_ad_onehot[i]))[:,1])
    vali_ad_rfrmse.append(rmse(vali_ad_click[i],vali_ad_rfpd[i]))
    vali_ad_rfauc.append(auc(vali_ad_click[i],vali_ad_rfpd[i]))
    print(i+1,'/9')
    

1 /9
2 /9
3 /9
4 /9
5 /9
6 /9
7 /9
8 /9
9 /9


In [17]:
j = 0
print('random forest trainning results')
for i in ad_name:
    
    print('ad',i,':','RMSE',train_ad_rfrmse[j],'AUC',train_ad_rfauc[j])
    
    j+=1
j = 0
print('random forest validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_rfrmse[j],'AUC',vali_ad_rfauc[j])
    j+=1

random forest trainning results
ad 1458 : RMSE 0.0206354988129 AUC 0.999407509158
ad 3476 : RMSE 0.0184468591658 AUC 0.999726397554
ad 3427 : RMSE 0.0224076299965 AUC 0.999052584637
ad 3358 : RMSE 0.021810827074 AUC 0.997375102623
ad 2259 : RMSE 0.0145293845256 AUC 0.999201421781
ad 2821 : RMSE 0.0215474213039 AUC 0.996983995637
ad 3386 : RMSE 0.0239921667852 AUC 0.996682757647
ad 2997 : RMSE 0.0654183671527 AUC 0.940485972623
ad 2261 : RMSE 0.0135280100227 AUC 0.999778775641
random forest validation results
ad 1458 : RMSE 0.0244789946604 AUC 0.915764902043
ad 3476 : RMSE 0.0180653417045 AUC 0.804952272268
ad 3427 : RMSE 0.0283000335803 AUC 0.94059983577
ad 3358 : RMSE 0.0263264345625 AUC 0.88220763778
ad 2259 : RMSE 0.0259315535327 AUC 0.614622578786
ad 2821 : RMSE 0.0250807669506 AUC 0.558995110478
ad 3386 : RMSE 0.0245902394732 AUC 0.724251378422
ad 2997 : RMSE 0.0658352288264 AUC 0.655449784902
ad 2261 : RMSE 0.0194210495672 AUC 0.680336700337


In [18]:
from sklearn.ensemble import ExtraTreesClassifier
etr_model = []
train_ad_etrpd = []
train_ad_etrrmse = []
train_ad_etrauc = []
vali_ad_etrpd = []
vali_ad_etrrmse = []
vali_ad_etrauc = []

for i in range(ad_len):
    etr_model.append(ExtraTreesClassifier(n_estimators=150, max_depth=20, random_state=2017))
    etr_model[i].fit(train_ad_onehot[i], train_ad_click[i])
    train_ad_etrpd.append((etr_model[i].predict_proba(train_ad_onehot[i]))[:,1])
    train_ad_etrrmse.append(rmse(train_ad_click[i],train_ad_etrpd[i]))
    train_ad_etrauc.append(auc(train_ad_click[i],train_ad_etrpd[i]))
    vali_ad_etrpd.append((etr_model[i].predict_proba(vali_ad_onehot[i]))[:,1])
    vali_ad_etrrmse.append(rmse(vali_ad_click[i],vali_ad_etrpd[i]))
    vali_ad_etrauc.append(auc(vali_ad_click[i],vali_ad_etrpd[i]))
    print(i+1,'/9')


1 /9
2 /9
3 /9
4 /9
5 /9
6 /9
7 /9
8 /9
9 /9


In [19]:
j = 0
print('extra tree regressor trainning results')
for i in ad_name:
    print('ad',i,':','RMSE',train_ad_etrrmse[j],'AUC',train_ad_etrauc[j])
    j+=1
    
j = 0
print('extra tree regressor validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_etrrmse[j],'AUC',vali_ad_etrauc[j])
    j+=1

extra tree regressor trainning results
ad 1458 : RMSE 0.0140260372712 AUC 0.999965001667
ad 3476 : RMSE 0.0126974736822 AUC 0.999998573914
ad 3427 : RMSE 0.0155911426269 AUC 0.99994270848
ad 3358 : RMSE 0.0163638245587 AUC 0.999198188183
ad 2259 : RMSE 0.0119800210738 AUC 0.999834014465
ad 2821 : RMSE 0.0176154045248 AUC 0.999669595998
ad 3386 : RMSE 0.0198284036356 AUC 0.999777033526
ad 2997 : RMSE 0.0635040482819 AUC 0.966523426171
ad 2261 : RMSE 0.0103079363546 AUC 0.999976241356
extra tree regressor validation results
ad 1458 : RMSE 0.0233972475267 AUC 0.92254039183
ad 3476 : RMSE 0.0181007358943 AUC 0.774514698715
ad 3427 : RMSE 0.0277463527719 AUC 0.939131039415
ad 3358 : RMSE 0.0257494361167 AUC 0.901073248498
ad 2259 : RMSE 0.0260401907863 AUC 0.582858694207
ad 2821 : RMSE 0.0253784219822 AUC 0.620409363289
ad 3386 : RMSE 0.0247225347456 AUC 0.677119984355
ad 2997 : RMSE 0.0668980301332 AUC 0.647255582301
ad 2261 : RMSE 0.019443100562 AUC 0.61872802095


In [20]:
from sklearn.ensemble import GradientBoostingClassifier
gbc_model = []
train_ad_gbcpd = []
train_ad_gbcrmse = []
train_ad_gbcauc = []
vali_ad_gbcpd = []
vali_ad_gbcrmse = []
vali_ad_gbcauc = []
    
for i in range(ad_len):

    gbc_model.append(GradientBoostingClassifier(n_estimators= 100,max_depth= 4, min_samples_leaf=15, learning_rate= 0.035))
#     print('fitting model:',i)
    gbc_model[i].fit(train_ad_onehot[i], train_ad_click[i])
#     print('predicting')
    vali_ad_gbcpd.append((gbc_model[i].predict_proba(vali_ad_onehot[i]))[:,1])
    vali_ad_gbcrmse.append(rmse(vali_ad_click[i],vali_ad_gbcpd[i]))
    vali_ad_gbcauc.append(auc(vali_ad_click[i],vali_ad_gbcpd[i]))
    print(i+1,'/9')

1 /9
2 /9
3 /9
4 /9
5 /9
6 /9
7 /9
8 /9
9 /9


In [21]:
for i in range(ad_len):
    train_ad_gbcpd.append((gbc_model[i].predict_proba(train_ad_onehot[i]))[:,1])

In [22]:
# j = 0
# print('extra tree classification trainning results')
# for i in ad_name:
#     print('ad',i,':','RMSE',train_ad_gbcrmse[j],'AUC',train_ad_gbcauc[j])
#     j+=1
j = 0
print('gradient boosting classification validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_gbcrmse[j],'AUC',vali_ad_gbcauc[j])
    j+=1

gradient boosting classification validation results
ad 1458 : RMSE 0.0269461885943 AUC 0.703915131305
ad 3476 : RMSE 0.0207309003167 AUC 0.674057629899
ad 3427 : RMSE 0.0282553662663 AUC 0.674674189447
ad 3358 : RMSE 0.0237282904513 AUC 0.920677388202
ad 2259 : RMSE 0.0259758503478 AUC 0.580102278268
ad 2821 : RMSE 0.0251567783722 AUC 0.592125039038
ad 3386 : RMSE 0.0245737886826 AUC 0.739048297085
ad 2997 : RMSE 0.0654336574265 AUC 0.69494200041
ad 2261 : RMSE 0.021365131573 AUC 0.572248410026


In [23]:
from sklearn.ensemble import AdaBoostClassifier
adb_model = []
train_ad_adbpd = []
train_ad_adbrmse = []
train_ad_adbauc = []
vali_ad_adbpd = []
vali_ad_adbrmse = []
vali_ad_adbauc = []

for i in range(ad_len):
    adb_model.append(AdaBoostClassifier(n_estimators=100, learning_rate=0.5, random_state=2017))
    adb_model[i].fit(train_ad_onehot[i], train_ad_click[i])
#     train_ad_adbpd.append((adb_model[i].predict_proba(train_ad_onehot[i]))[:,1])
#     train_ad_adbrmse.append(rmse(train_ad_click[i],train_ad_adbpd[i]))
#     train_ad_adbauc.append(auc(train_ad_adbpd[i],train_ad_adbpd[i]))
    vali_ad_adbpd.append((adb_model[i].predict_proba(vali_ad_onehot[i]))[:,1])
    vali_ad_adbrmse.append(rmse(vali_ad_click[i],vali_ad_adbpd[i]))
    vali_ad_adbauc.append(auc(vali_ad_click[i],vali_ad_adbpd[i]))
    print(i+1,'/9')


1 /9
2 /9
3 /9
4 /9
5 /9
6 /9
7 /9
8 /9
9 /9


In [24]:
# j = 0
# print('Adaboosting trainning results')
# for i in ad_name:
#     print('ad',i,':','RMSE',train_ad_adbrmse[j],'AUC',train_ad_adbauc[j])
#     j+=1
j = 0
print('Adaboosting validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_adbrmse[j],'AUC',vali_ad_adbauc[j])
    j+=1

Adaboosting validation results
ad 1458 : RMSE 0.450230607151 AUC 0.96234731138
ad 3476 : RMSE 0.448387021958 AUC 0.685262452995
ad 3427 : RMSE 0.453616844208 AUC 0.949467357138
ad 3358 : RMSE 0.450010233814 AUC 0.937913745946
ad 2259 : RMSE 0.417785497635 AUC 0.585811799122
ad 2821 : RMSE 0.452874823988 AUC 0.601711332761
ad 3386 : RMSE 0.459623950185 AUC 0.695428534388
ad 2997 : RMSE 0.466812336247 AUC 0.674645344669
ad 2261 : RMSE 0.403476709359 AUC 0.565679012346


In [25]:
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
pipe_model = []

for i in range(ad_len):
    pipe_model.append(Pipeline([('feature_selection', SelectFromModel(rf_model[i])),('classification',rf_model[i])]))
    pipe_model[i].fit(train_ad_onehot[i],train_ad_click[i])

In [26]:
pipe_pd = []
pipe_rmse = []
pipe_auc = []
for i in range(ad_len):
    pipe_pd.append((pipe_model[i].predict_proba(vali_ad_onehot[i]))[:,1])
    pipe_rmse.append(rmse(vali_ad_click[i],pipe_pd[i]))
    pipe_auc.append(auc(vali_ad_click[i],pipe_pd[i]))

In [27]:
j = 0
print('pipe+rf validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',pipe_rmse[j],'AUC',pipe_auc[j])
    j+=1

pipe+rf validation results
ad 1458 : RMSE 0.0225445415203 AUC 0.936548228429
ad 3476 : RMSE 0.0182073395726 AUC 0.759756546961
ad 3427 : RMSE 0.0266955381163 AUC 0.932587677474
ad 3358 : RMSE 0.025772788801 AUC 0.900473775378
ad 2259 : RMSE 0.0259403996502 AUC 0.633191680333
ad 2821 : RMSE 0.0251815299869 AUC 0.60585547119
ad 3386 : RMSE 0.0246136344906 AUC 0.721710973374
ad 2997 : RMSE 0.067272112364 AUC 0.620944509884
ad 2261 : RMSE 0.0194772234451 AUC 0.378174335952


In [28]:
# ## Concatenating Level 1 results
train_L1 = []
test_L1 = []
L2_model = []
vali_pd = []
for i in range(ad_len):
    print(i)
    train_L1.append(np.column_stack((train_ad_gbcpd[i],train_ad_rfpd[i],train_ad_pd[i])))
    test_L1.append(np.column_stack((vali_ad_gbcpd[i],vali_ad_rfpd[i],vali_ad_pd[i])))#add as much as u want and wish u good luck
    

    # ## Fitting Level 1 models

    L2_model.append(RandomForestClassifier(n_estimators=66, max_depth=16, random_state=1024))#y_pred_rfbagging,y_pred_xgb

    L2_model[i].fit(train_L1[i],train_ad_click[i])
    vali_pd.append(L2_model[i].predict(test_L1[i]))

0
1
2
3
4
5
6
7
8


In [29]:
st_vali_pd = []
for i in range(ad_len):
    st_vali_pd.append((L2_model[i].predict_proba(test_L1[i]))[:,1])

In [30]:
for i in range(ad_len):
    print('RMSE',rmse(vali_ad_click[i],st_vali_pd[i]),'AUC',auc(vali_ad_click[i],st_vali_pd[i]))

RMSE 0.0269302627164 AUC 0.895781909129
RMSE 0.0199903453799 AUC 0.627741521344
RMSE 0.0303891607233 AUC 0.77827753055
RMSE 0.0270759497991 AUC 0.748761088884
RMSE 0.0275496601772 AUC 0.486439541687
RMSE 0.0291799672515 AUC 0.480600503592
RMSE 0.0292834039277 AUC 0.546990214675
RMSE 0.0738400631875 AUC 0.560592287207
RMSE 0.0199694437269 AUC 0.460044893378


In [31]:
from sklearn.linear_model import Lasso

lasso_model = []
train_ad_lassopd = []
train_ad_lassormse = []
train_ad_lassoauc = []
vali_ad_lassopd = []
vali_ad_lassormse = []
vali_ad_lassoauc = []

for i in range(ad_len):
    lasso_model.append(Lasso(alpha=0.00004))
    lasso_model[i].fit(train_ad_onehot[i], train_ad_click[i])
    train_ad_lassopd.append((lasso_model[i].predict(train_ad_onehot[i])))
    train_ad_lassormse.append(rmse(train_ad_click[i],train_ad_lassopd[i]))
    train_ad_lassoauc.append(auc(train_ad_click[i],train_ad_lassopd[i]))
    vali_ad_lassopd.append((lasso_model[i].predict(vali_ad_onehot[i])))
    vali_ad_lassormse.append(rmse(vali_ad_click[i],vali_ad_lassopd[i]))
    vali_ad_lassoauc.append(auc(vali_ad_click[i],vali_ad_lassopd[i]))
    print(i+1,'/9')

1 /9
2 /9
3 /9
4 /9
5 /9
6 /9
7 /9
8 /9
9 /9


In [32]:
j = 0
print('Lasso validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_lassormse[j],'AUC',vali_ad_lassoauc[j])
    j+=1

Lasso validation results
ad 1458 : RMSE 0.024319230495 AUC 0.889207836599
ad 3476 : RMSE 0.0182846412984 AUC 0.797896136213
ad 3427 : RMSE 0.0297063306345 AUC 0.906300769937
ad 3358 : RMSE 0.0278221759976 AUC 0.897813647621
ad 2259 : RMSE 0.0258764216646 AUC 0.644292141306
ad 2821 : RMSE 0.0249758353338 AUC 0.585621535369
ad 3386 : RMSE 0.0244187993159 AUC 0.727662755943
ad 2997 : RMSE 0.0654280174078 AUC 0.678118278193
ad 2261 : RMSE 0.0193342687271 AUC 0.773310886644


In [35]:
from sklearn.linear_model import Ridge

ridge_model = []
train_ad_ridgepd = []
train_ad_ridgermse = []
train_ad_ridgeauc = []
vali_ad_ridgepd = []
vali_ad_ridgermse = []
vali_ad_ridgeauc = []
for i in range(ad_len):
    ridge_model.append(Ridge(alpha=0))
    ridge_model[i].fit(train_ad_onehot[i], train_ad_click[i])
    train_ad_ridgepd.append((ridge_model[i].predict(train_ad_onehot[i])))
    train_ad_ridgermse.append(rmse(train_ad_click[i],train_ad_ridgepd[i]))
    train_ad_ridgeauc.append(auc(train_ad_click[i],train_ad_ridgepd[i]))
    vali_ad_ridgepd.append((ridge_model[i].predict(vali_ad_onehot[i])))
    vali_ad_ridgermse.append(rmse(vali_ad_click[i],vali_ad_ridgepd[i]))
    vali_ad_ridgeauc.append(auc(vali_ad_click[i],vali_ad_ridgepd[i]))
    print(i+1,'/9')



1 /9
2 /9
3 /9
4 /9
5 /9
6 /9
7 /9
8 /9
9 /9


In [36]:
j = 0
print('ridge validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_ridgermse[j],'AUC',vali_ad_ridgeauc[j])
    j+=1

ridge validation results
ad 1458 : RMSE 0.0241945485354 AUC 0.908487036265
ad 3476 : RMSE 0.0182992712889 AUC 0.787690841585
ad 3427 : RMSE 0.0296858404349 AUC 0.898131004097
ad 3358 : RMSE 0.0277765753306 AUC 0.872620995027
ad 2259 : RMSE 0.0258769773153 AUC 0.57909390098
ad 2821 : RMSE 0.0249785769926 AUC 0.669823059806
ad 3386 : RMSE 0.0244054155969 AUC 0.728055531293
ad 2997 : RMSE 0.0654387042822 AUC 0.67899531394
ad 2261 : RMSE 0.019349782198 AUC 0.50910587355


In [37]:
from sklearn.linear_model import ElasticNet

et_model = []
train_ad_etpd = []
train_ad_etrmse = []
train_ad_etauc = []
vali_ad_etpd = []
vali_ad_etrmse = []
vali_ad_etauc = []

for i in range(ad_len):
    et_model.append(ElasticNet(alpha=0.000008, l1_ratio=0.7))
    et_model[i].fit(train_ad_onehot[i], train_ad_click[i])
    train_ad_etpd.append((et_model[i].predict(train_ad_onehot[i])))
    train_ad_etrmse.append(rmse(train_ad_click[i],train_ad_etpd[i]))
    train_ad_etauc.append(auc(train_ad_click[i],train_ad_etpd[i]))
    vali_ad_etpd.append((et_model[i].predict(vali_ad_onehot[i])))
    vali_ad_etrmse.append(rmse(vali_ad_click[i],vali_ad_etpd[i]))
    vali_ad_etauc.append(auc(vali_ad_click[i],vali_ad_etpd[i]))
    print(i+1,'/9')

1 /9
2 /9
3 /9
4 /9
5 /9
6 /9
7 /9
8 /9
9 /9


In [38]:
j = 0
print('elastic net validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_etrmse[j],'AUC',vali_ad_etauc[j])
    j+=1

elastic net validation results
ad 1458 : RMSE 0.0242135117801 AUC 0.901769904127
ad 3476 : RMSE 0.0182905485456 AUC 0.779818440458
ad 3427 : RMSE 0.0296844008066 AUC 0.904117441902
ad 3358 : RMSE 0.0277655410721 AUC 0.879011542732
ad 2259 : RMSE 0.0258758508431 AUC 0.625645472275
ad 2821 : RMSE 0.0249759824242 AUC 0.667998028576
ad 3386 : RMSE 0.0244061613161 AUC 0.732153304339
ad 2997 : RMSE 0.0654299443657 AUC 0.680634154461
ad 2261 : RMSE 0.0193370816714 AUC 0.5815114104


In [39]:
best = [0.964258274281,0.823949256842,0.952029031574,0.937509717487,0.644292141306,0.674527150999,0.729408271574,0.661435393834,0.773310886644]

In [40]:
list(map(lambda x: x[0]-x[1], zip(best, vali_ad_etauc)))

[0.062488370154280548,
 0.044130816383782245,
 0.047911589672467225,
 0.058498174755242105,
 0.018646669030834784,
 0.0065291224231098788,
 -0.0027450327653710804,
 -0.019198760626719102,
 0.19179947624370075]

# Save Results

In [41]:
files = ['xgb_result','rf_result','etc_result','gbc_result','adb_result','Lasso','ridge','et']

In [42]:
def result_save (pd, auc, rmse):
    result = []
    result.append(pd)
    result.append(auc)
    result.append(rmse)
    
    return result

In [43]:
#bag_model = [rf_bag, etr_bag, gbc_bag, adb_bag] 
xgb_result = result_save (vali_ad_pd, vali_ad_auc, vali_ad_rmse,)
rf_result = result_save (vali_ad_rfpd, vali_ad_rfauc, vali_ad_rfrmse)
# rf_bag_result = result_save (bag_prob[0],bag_auc[0],bag_rmse[0])
etc_result = result_save (vali_ad_etrpd, vali_ad_etrauc, vali_ad_etrrmse)
# etc_bag_result = result_save (bag_prob[1],bag_auc[1],bag_rmse[1])
gbc_result = result_save (vali_ad_gbcpd, vali_ad_gbcauc, vali_ad_gbcrmse)
# gbc_bag_result = result_save (bag_prob[2],bag_auc[2],bag_rmse[2])
adb_result = result_save (vali_ad_rfpd, vali_ad_rfauc, vali_ad_rfrmse)
# adb_bag_result = result_save (bag_prob[3],bag_auc[3],bag_rmse[3])
lasso_result = result_save (vali_ad_lassopd, vali_ad_lassoauc, vali_ad_lassormse)
ridge_result = result_save (vali_ad_ridgepd, vali_ad_ridgeauc, vali_ad_ridgermse)
et_result = result_save (vali_ad_etpd, vali_ad_etauc, vali_ad_etrmse)

In [44]:
result_list = [xgb_result,rf_result,etc_result,gbc_result,adb_result,lasso_result,ridge_result,et_result]

In [45]:
import pickle
j = 0
for i in files:
    with open(i+".txt", "wb") as fp:
        pickle.dump(result_list[j], fp)
    j+=1

In [46]:
model = [xgb_model,rf_model,etr_model,gbc_model,adb_model,lasso_model,ridge_model,et_model]

In [47]:
# xgb xgb xgb gbc lasso rf rf xgb lasso
no = [0, 0, 0, 3, 5, 1, 1, 0, 5]

In [48]:
combine_model = []
j = 0
for i in no:
    combine_model.append(model[i][j])
    j+=1

In [51]:
creative_list = train['creative'].value_counts().index.tolist()[:35]
def test_data(data):
    data['weekday'].apply(str)
    data['hour'].apply(str)
    
    os_name = ['windows','android','mac','other','linux','ios']
    data["OS"] = float('NaN')
    j=0
    for i in (os_name):
        data["OS"][data['useragent'].map(lambda x: x.split('_')[0])==i] = os_name[j]
        j +=1
    
    data["creative_new"] = 'others'
    j=0
    for i in creative_list:
        data["creative_new"][data['creative']==i] = creative_list[j]
        j +=1
    
    browser_name = ['ie','chrome','safari','firefox','other','theworld','maxthon','opera','sogou']
    data["Browser"] = float('NaN')
    j=0
    for i in (browser_name):
        data["Browser"][data['useragent'].map(lambda x: x.split('_')[1])==i] = browser_name[j]
        j +=1
    
    data["IPaddress"] = float('NaN')
    data["IPaddress"][data["IP"].map(lambda x: x.split('.')[0]).astype(float)<= 255]='E'
    data["IPaddress"][data['IP'].map(lambda x: x.split('.')[0]).astype(float)<= 239]='D'
    data["IPaddress"][data['IP'].map(lambda x: x.split('.')[0]).astype(float)<= 223]='C'
    data["IPaddress"][data['IP'].map(lambda x: x.split('.')[0]).astype(float)<= 191]='B'
    data["IPaddress"][data['IP'].map(lambda x: x.split('.')[0]).astype(float)<= 126]='A'
    
    data["slotsize"]= data.slotwidth.astype(str).str.cat(data.slotheight.astype(str), sep='*')
    
    data["slotprice_range"] = 0
    data["slotprice_range"][data["slotprice"]>0]='1-10'
    data["slotprice_range"][data["slotprice"]>10]='11-50'
    data["slotprice_range"][data["slotprice"]>50]='51-100'
    data["slotprice_range"][data["slotprice"]>100]='101+'
    
    data['slotvisibility_s']=data["slotvisibility"]
    data['slotvisibility_s'][data["slotvisibility"]=='0']='Na'
    data['slotvisibility_s'][data["slotvisibility"]=='1']='FirstView'
    data['slotvisibility_s'][data["slotvisibility"]=='2']='SecondView'
    
    data["platform"]='other'
    PC_name=['windows','mac','linux']
    phone_name=['android','ios']

    for i in (PC_name):
        data['platform'][data["OS"]==i] = 'PC'

    for i in (phone_name):
        data['platform'][data["OS"]==i] = 'phone'
        
    usertag = data['usertag']
    aaa = usertag.str.get_dummies(sep = ',')
    data = data.drop('usertag',1)
    data = data.drop(['logtype','slotprice','bidid','userid','useragent','domain','IP','url','urlid','city','slotid','slotwidth','slotheight','creative'],axis=1)
    data=data.applymap(str)
    bbb = pd.get_dummies(data)
    onehot = pd.concat([bbb,aaa],axis = 1, join_axes = [bbb.index])
    
    
    return onehot

In [85]:
def sq(pctr, basectr, basebid):
    price = int(pctr*pctr*4/basectr *basebid)
    return price

In [81]:
def ad_to_total (ad_onehot,ad_pd):
    ad_pctr = ad_onehot
    j = 0
    for i in ad_name:
        ad_pctr[j]['pctr'] = ad_pd[j]
        ad_pctr[j]['ad'] = i
        ad_pctr[j] = ad_pctr[j][['pctr','ad']]
        j+=1
    pctr = (pd.concat(ad_pctr)).sort_index()
    return pctr

In [104]:
import time
start_time = time.time()
test_onehot = test_data(test)
ad_onehot = []
for i in ad_name:
    ad_onehot.append(test_onehot[test_onehot["advertiser_"+str(i)]==1]) 
test_pd = [0]*9
for i in [0,1,2,7]:
    dpvali =xgb.DMatrix(ad_onehot[i])
    test_pd[i] = (combine_model[i].predict(dpvali))
for i in [4,8]:
    test_pd[i] = combine_model[i].predict(ad_onehot[i])
test_pd[5] = rf_model[5].predict(ad_onehot[5])
test_pd[6] = rf_model[6].predict(ad_onehot[6])

test_pctr = ad_to_total(ad_onehot,test_pd)

test_price = []
for index in range(len(test_pctr)):
    adid = test['advertiser'].iloc[index]    
    basectr = ad_ctr[ad_name.index(adid)]
    pctr = test_pctr['pctr'].iloc[index]
    test_price.append(sq(pctr,basectr,72000))
time = time.time()-start_time
time = time/len(test)
time

0.00011279020403018402

In [99]:
test_result = pd.DataFrame({'bidid':test['bidid'],'bid_price':test_price},columns = ['bidid','bid_price'])

In [100]:
test_result.head()

,bidid,bid_price
0,fe2e06dff1dfd227471fd1ca717888ac,240
1,92ea762b7b2cfd7aee30a2e88e1f5899,1
2,617dfab714eedf4988c4aa9ad5425eca,0
3,7fa016aaa3db39413b482f757facc3f4,0
4,49db062ac10d19591dc7d988d730aad5,25


In [101]:
test_result.to_csv('test_bidding_price.csv',index = False)

In [102]:
pd.read_csv('test_bidding_price.csv').head()

,bidid,bid_price
0,fe2e06dff1dfd227471fd1ca717888ac,240
1,92ea762b7b2cfd7aee30a2e88e1f5899,1
2,617dfab714eedf4988c4aa9ad5425eca,0
3,7fa016aaa3db39413b482f757facc3f4,0
4,49db062ac10d19591dc7d988d730aad5,25
